# jsonからcsvに解答データをコンバート

In [ ]:
import json
import pandas as pd
import re
from statistics import mode

def process_text(text):
    if isinstance(text, list):
        text = ' '.join(text)
    text = text.lower()
    num_word_to_digit = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'ten': '10'
    }
    for word, digit in num_word_to_digit.items():
        text = text.replace(word, digit)
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)
    text = re.sub(r'\b(a|an|the)\b', '', text)
    contractions = {
        "dont": "don't", "isnt": "isn't", "arent": "aren't", "wont": "won't",
        "cant": "can't", "wouldnt": "wouldn't", "couldnt": "couldn't"
    }
    for contraction, correct in contractions.items():
        text = text.replace(contraction, correct)
    text = re.sub(r"[^\w\s':]", ' ', text)
    text = re.sub(r'\s+,', ',', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def clean_text(text):
    if isinstance(text, tuple):
        text = text[0]
    text = re.sub(r"[\(\)\"\',]", '', text)
    return text

class VQADatasetProcessor:
    def __init__(self, df_path,answer):
        self.df = pd.read_json(df_path, convert_axes=False)
        self.is_train = answer
    
    def process_data(self):
        processed_data = []
        for idx in range(len(self.df)):
            question_text = clean_text(process_text(self.df["question"].iloc[idx]))
            if self.is_train!=False: 
                answers = [process_text(answer["answer"]) for answer in self.df["answers"].iloc[idx]]
                if len(answers) > 0:
                    mode_answer = mode(answers)
                    answer_text = clean_text(str(mode_answer))
                else:
                    answer_text = "unanswerable"
                processed_data.append({
                    'image': self.df['image'].iloc[idx],
                    'question': question_text,
                    'answer': answer_text
                })
            else:
                processed_data.append({
                    'image': self.df['image'].iloc[idx],
                    'question': question_text,
                })
        return processed_data

def preprocess_and_save(df_path, output_csv,answer =False):
    processor = VQADatasetProcessor(df_path,answer)
    processed_data = processor.process_data()
    df = pd.DataFrame(processed_data)
    df.to_csv(output_csv, index=False)

# 前処理を実行してCSVに保存
preprocess_and_save('./data/valid.json', './data/processed_valid.csv',False)


# データの抽出

In [1]:
import pandas as pd

# ファイルの読み込み
file_path = './data/processed_train.csv'
data = pd.read_csv(file_path)

# unanswerableの行を75個ランダムに抽出
unanswerable_data = data[data['answer'] == 'unanswerable'].sample(n=100, random_state=1)

# それ以外の行を125個ランダムに抽出
answerable_data = data[data['answer'] != 'unanswerable'].sample(n=300, random_state=1)

# データを結合
sampled_data = pd.concat([unanswerable_data, answerable_data])

# 新しいCSVファイルとして保存
output_path = './data/extracted_train.csv'
sampled_data.to_csv(output_path, index=False)



# 画像データの適応的ヒストグラム平坦化

In [ ]:
import cv2
import os
import numpy as np

def apply_clahe_to_color_image(image_path, output_path):
    # 画像をカラーで読み込む
    image = cv2.imread(image_path)
    
    # 画像をLAB色空間に変換
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    
    # LAB画像を3つのチャンネルに分割
    l, a, b = cv2.split(lab)
    
    # CLAHEオブジェクトを作成（クリップ制限を設定）
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    
    # LチャンネルにCLAHEを適用
    cl = clahe.apply(l)
    
    # CLAHE適用後のLチャンネルと元のA、Bチャンネルを統合
    limg = cv2.merge((cl, a, b))
    
    # LAB色空間からBGR色空間に変換
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    
    # 画像を保存
    cv2.imwrite(output_path, final)

def process_images_in_folder(input_folder, output_folder):
    # 入力フォルダ内のすべてのファイルを取得
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            
            # CLAHEを適用
            apply_clahe_to_color_image(input_path, output_path)
            print(f"Processed {filename}")

# 入力フォルダと出力フォルダを指定
input_folder = "../data/train"
output_folder = "../data/processed_train"

# 出力フォルダが存在しない場合は作成
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# フォルダ内の画像を処理
process_images_in_folder(input_folder, output_folder)


In [8]:
import cv2
import os
import numpy as np

def apply_clahe_to_color_image(image_path, output_path):
    # 画像をカラーで読み込む
    image = cv2.imread(image_path)
    
    # 画像をLAB色空間に変換
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    
    # LAB画像を3つのチャンネルに分割
    l, a, b = cv2.split(lab)
    
    # CLAHEオブジェクトを作成（クリップ制限を設定）
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    
    # LチャンネルにCLAHEを適用
    cl = clahe.apply(l)
    
    # CLAHE適用後のLチャンネルと元のA、Bチャンネルを統合
    limg = cv2.merge((cl, a, b))
    
    # LAB色空間からBGR色空間に変換
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    
    # 画像をシャープ化する
    sharpened_image = sharpen_image(final)
    
    # 画像を保存
    cv2.imwrite(output_path, sharpened_image)

def sharpen_image(image):
    # シャープ化のためのカーネルを定義
    # kernel = np.array([[ 0, -0.5,  0],
    #                [-0.5,  4, -0.5],
    #                [ 0, -0.5,  0]])
    kernel = np.array([[ 0, -1,  0],
                   [-1,  5, -1],
                   [ 0, -1,  0]])

    # フィルタを適用して画像をシャープ化
    sharpened_image = cv2.filter2D(image, -1, kernel)
    return sharpened_image

def process_images_in_folder(input_folder, output_folder):
    # 入力フォルダ内のすべてのファイルを取得
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            
            # CLAHEとシャープ化を適用
            apply_clahe_to_color_image(input_path, output_path)
            print(f"Processed {filename}")

# 入力フォルダと出力フォルダを指定
input_folder = "../data/valid"
output_folder = "../data/shraped_valid"

# 出力フォルダが存在しない場合は作成
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# フォルダ内の画像を処理
process_images_in_folder(input_folder, output_folder)


Processed valid_03541.jpg
Processed valid_02779.jpg
Processed valid_00103.jpg
Processed valid_02631.jpg
Processed valid_01793.jpg
Processed valid_04080.jpg
Processed valid_00561.jpg
Processed valid_02393.jpg
Processed valid_03821.jpg
Processed valid_02777.jpg
Processed valid_02407.jpg
Processed valid_04765.jpg
Processed valid_03449.jpg
Processed valid_01153.jpg
Processed valid_01849.jpg
Processed valid_03953.jpg
Processed valid_04906.jpg
Processed valid_03676.jpg
Processed valid_01925.jpg
Processed valid_04570.jpg
Processed valid_00015.jpg
Processed valid_04354.jpg
Processed valid_02145.jpg
Processed valid_00346.jpg
Processed valid_03061.jpg
Processed valid_00860.jpg
Processed valid_03237.jpg
Processed valid_02350.jpg
Processed valid_02728.jpg
Processed valid_04203.jpg
Processed valid_01555.jpg
Processed valid_02593.jpg
Processed valid_02029.jpg
Processed valid_02218.jpg
Processed valid_03969.jpg
Processed valid_03693.jpg
Processed valid_00954.jpg
Processed valid_04073.jpg
Processed va

KeyboardInterrupt: 